In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
import random
from sklearn.svm import OneClassSVM

# Imbalanced learn dataset

In [2]:
from imblearn.datasets import fetch_datasets
fetch_dataset=fetch_datasets()
#fetch_dataset=readfile("fetch_data.pkl")
data_name=[]
for word in fetch_dataset:
    print(word)
    data_name.append(word)

ecoli
optical_digits
satimage
pen_digits
abalone
sick_euthyroid
spectrometer
car_eval_34
isolet
us_crime
yeast_ml8
scene
libras_move
thyroid_sick
coil_2000
arrhythmia
solar_flare_m0
oil
car_eval_4
wine_quality
letter_img
yeast_me2
webpage
ozone_level
mammography
protein_homo
abalone_19


# Traditional OCSVM

In [ ]:

AUC_pos_classic=[]
#AUC_neg_all=[]
for e in range(5):
    AUC_pos=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        
        OCSVM=OneClassSVM()
        OCSVM.fit(positive)
        scores=OCSVM.score_samples(X_test)
        AUC_pos.append(roc_auc_score(y_test,scores))
        print(roc_auc_score(y_test,scores))
        
    AUC_pos_classic.append(AUC_pos)
    #AUC_neg_all.append(AUC_neg)

In [ ]:
%%time
AUC_neg_classic=[]
#AUC_neg_all=[]
for e in range(5):
    AUC_neg=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        
        OCSVM=OneClassSVM()
        OCSVM.fit(negative)
        scores=OCSVM.score_samples(X_test)
        AUC_neg.append(roc_auc_score(y_test,-scores))
        print(roc_auc_score(y_test,-scores))
        
    AUC_neg_classic.append(AUC_neg)
    #AUC_neg_all.append(AUC_neg)

# ODSVM-all

In [ ]:
AUC_pos_all=[]
#AUC_neg_all=[]
for e in range(5):
    AUC_pos=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        models=[]
        for data in positive:
            OCSVM=OneClassSVM()
            OCSVM.fit(data.reshape(1,-1))
            models.append(OCSVM)
        ens=np.zeros(len(X_test))
        for model in models:
            ens+=model.score_samples(X_test)
        AUC_pos.append(roc_auc_score(y_test,ens))
        print(roc_auc_score(y_test,ens))
        
    AUC_pos_all.append(AUC_pos)
    #AUC_neg_all.append(AUC_neg)

In [ ]:
%%time
AUC_neg_all=[]
#AUC_neg_all=[]
for e in range(5):
    AUC_neg=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        
        models=[]
        for data in negative:
            OCSVM=OneClassSVM()
            OCSVM.fit(data.reshape(1,-1))
            models.append(OCSVM)
        ens=np.zeros(len(X_test))
        for model in models:
            ens+=model.score_samples(X_test)
        AUC_neg.append(roc_auc_score(y_test,-ens))
        print(roc_auc_score(y_test,-ens))
        
    AUC_neg_all.append(AUC_neg)
    #AUC_neg_all.append(AUC_neg)

# ODSVM-N

In [ ]:
AUC_pos_fast=[]

#AUC_base=[]
for e in range(5):
    AUC_pos=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        models=[]
        avg=positive.mean(axis=0)

        OCSVM=OneClassSVM()
        OCSVM.fit(avg.reshape(1,-1))
        #models.append(OCSVM)
        first=OCSVM.score_samples(positive)
        index=first.argsort().argsort()
        #Set some N
        N=200
        support=positive[index<N]
    
        
        for data in support:
            OCSVM=OneClassSVM()
            OCSVM.fit(data.reshape(1,-1))
            models.append(OCSVM)
                
        
        ens=np.zeros(len(X_test))
        #num_base=[]
        #base=[]
        for model in models:
            ens+=model.score_samples(X_test)
            #num_base.append(roc_auc_score(y_test,model.score_samples(X_test)))
            #base.append(model.score_samples(X_test))
        #ens=np.array(base).max(axis=0)   
        #AUC_base.append(num_base)
        #print(len(models))
        AUC_pos.append(roc_auc_score(y_test,ens))
        #print(roc_auc_score(y_test,ens))
        
    AUC_pos_fast.append(AUC_pos)

In [7]:
AUC_pos_fast=np.array(AUC_pos_fast)

In [8]:
AUC_pos_fast

array([[0.89020071, 0.97478278, 0.9016833 , 0.42085904, 0.79336549,
        0.69305156, 0.63162393, 0.98728666, 0.93346938, 0.64665763,
        0.60593247, 0.59181772, 0.93880597, 0.73831733, 0.52332848,
        0.56783626, 0.4699993 , 0.56493733, 0.99127974, 0.43835616,
        0.88841763, 0.78310105, 0.61659069, 0.78282157, 0.80201989,
        0.86797232, 0.69012712],
       [0.93683589, 0.98088587, 0.89554905, 0.42406093, 0.8217991 ,
        0.68817924, 0.43874644, 0.98220713, 0.93995311, 0.71248871,
        0.58202339, 0.61405766, 0.88208955, 0.71548127, 0.54849068,
        0.66959064, 0.53868235, 0.52506964, 0.99566875, 0.47103106,
        0.88534466, 0.83057491, 0.61205197, 0.75358467, 0.79411195,
        0.87558033, 0.65950636],
       [0.90023613, 0.97143442, 0.90080379, 0.38271066, 0.80925785,
        0.66629473, 0.57806268, 0.98885406, 0.92353248, 0.7346206 ,
        0.55858589, 0.62301048, 0.87910448, 0.72363536, 0.55225646,
        0.52339181, 0.64716796, 0.67635794, 0.9958

In [ ]:
AUC_neg_fast=[]
AUC_base_neg=[]
#AUC_neg_all=[]
for e in range(5):
    AUC_neg=[]
    #AUC_neg=[]
    for word in fetch_dataset:
        #print(word)
        X=fetch_dataset[word]["data"]
        Y=fetch_dataset[word]["target"]
        #print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
        X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        X[pd.isnull(X)]=0
        random.seed(0)
        d=len(X[0])
        split=0.4
        #random_value=1
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
        positive=X_train[y_train==1]
        negative=X_train[y_train==-1]
        
        models=[]
        avg=negative.mean(axis=0)

        OCSVM=OneClassSVM()
        OCSVM.fit(avg.reshape(1,-1))
        #models.append(OCSVM)
        first=OCSVM.score_samples(negative)
        index=first.argsort().argsort()
        
        #OCSVM=OneClassSVM()
        
        #OCSVM.fit(negative[index].reshape(1,-1))
        #models.append(OCSVM)
        
        #scores=OCSVM.score_samples(negative)

        #change N
        N=1000
        support=negative[index<N]
        #index=index[index<1000]
        #support=support[index.argsort()]
        
        for data in support:
            OCSVM=OneClassSVM()
            OCSVM.fit(data.reshape(1,-1))
            models.append(OCSVM)
                
        
        ens=np.zeros(len(X_test))
        #num_base=[]
        for model in models:
            ens+=model.score_samples(X_test)
            #num_base.append(roc_auc_score(y_test,-model.score_samples(X_test)))

        #AUC_base_neg.append(num_base)    
        #ens=np.array(num_base).max(axis=0)
        #print(len(models))
        AUC_neg.append(roc_auc_score(y_test,-ens))
        #print(roc_auc_score(y_test,-ens))
        
    AUC_neg_fast.append(AUC_neg)
    #AUC_neg_all.append(AUC_neg)